In [31]:
from pandas import read_csv
import pandas as pd
import plotly.graph_objects as go
import numpy as np
#Load data set
look_back = 2
series_influ_A_df = read_csv('../temp_data/influA_vietnam_last_10_days.csv', engine='python')
series_influ_A_df = series_influ_A_df[["Day","Influenza A - All types of surveillance", "temp", "tempmax", "dew","windspeed"]]
series_influ_A_df['Day'] = pd.to_datetime(series_influ_A_df['Day'])
import pandas as pd
df = series_influ_A_df.rename(columns= {"Influenza A - All types of surveillance": "case"})
train_df = df.iloc[0:]

In [32]:

def plot_month_per_year(df, key):
    import plotly.graph_objects as go
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Day'], y=df[key], mode='lines+markers', name=f'Influenza A Cases in month'))
    fig.update_layout(
        title='Influenza A Cases Over Time',
        # xaxis=dict(
        #     title='Day',
        #     tickvals=df['Day'],  
        #     tickformat='%Y-%m',
        #     tickangle=45,
        # ),
        yaxis=dict(title='Number of Cases'),
        showlegend=True,
    )
 
    # Show plot
    fig.show()

In [33]:
plot_month_per_year(train_df, 'case')

In [34]:
import plotly.express as px

fig = px.histogram(train_df, x='case')
fig.show()

# skew data , not gaussian distribution => should IQR to determine outliers

In [35]:
#create a box plot

fig = px.box(train_df, y='case')

fig.show()

In [36]:
def find_outliers_IQR(df):

    q1=df.quantile(0.25)

    q3=df.quantile(0.75)

    IQR=q3-q1

    outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

    return outliers

In [37]:
outliers = find_outliers_IQR(train_df['case'])

print('number of outliers: '+ str(len(outliers)))

print('max outlier value: '+ str(outliers.max()))

print('min outlier value: '+ str(outliers.min()))

number of outliers: 47
max outlier value: 82.0
min outlier value: 29.0


In [38]:
def drop_outliers_IQR(df, key):

    q1=df[key].quantile(0.25)

    q3=df[key].quantile(0.75)

    IQR=q3-q1

    not_outliers = df[~((df[key]<(q1-1.5*IQR)) | (df[key]>(q3+1.5*IQR)))]

    outliers_dropped = not_outliers.dropna().reset_index()

    return outliers_dropped

outliers_dropped = drop_outliers_IQR(train_df, 'case')
plot_month_per_year(outliers_dropped, 'case')


In [39]:
def impute_outliers_IQR(df):

    q1=df.quantile(0.25)

    q3=df.quantile(0.75)

    IQR=q3-q1

    upper = df[~(df>(q3+1.5*IQR))].max()

    lower = df[~(df<(q1-1.5*IQR))].min()

    df = np.where(
        df > upper,
        int(df.mean()),
        np.where(df < lower,int(df.mean()),df)
        )
    return df
impute_df = train_df.copy()
impute_df['case'] = impute_outliers_IQR(train_df['case'])
plot_month_per_year(impute_df, 'case')

In [40]:
impute_df.to_csv('../temp_data/remove_outlier_impute.csv', index=0)